In [ ]:
import logging
import pathlib
import sys
from itertools import product

import pandas as pd
from pandas import DataFrame
from tqdm import tqdm

from topollm.analysis.compare_sampling_methods.filter_dataframe_based_on_filters_dict import (
    filter_dataframe_based_on_filters_dict,
)
from topollm.analysis.compare_sampling_methods.load_and_concatenate_saved_dataframes import (
    load_and_concatenate_saved_dataframes,
)
from topollm.config_classes.constants import NAME_PREFIXES_TO_FULL_DESCRIPTIONS, TOPO_LLM_REPOSITORY_BASE_PATH
from topollm.typing.enums import Verbosity

# Create a logger
default_logger: logging.Logger = logging.getLogger(name=__name__)
default_logger.setLevel(level=logging.DEBUG)

# Create a stream handler
stream_handler = logging.StreamHandler(stream=sys.stdout)
stream_handler.setLevel(level=logging.DEBUG)

# Create a formatter and attach it to the handler
formatter = logging.Formatter(fmt="[%(asctime)s][%(levelname)8s][%(name)s] %(message)s (%(filename)s:%(lineno)s)")
stream_handler.setFormatter(fmt=formatter)

# Add the handler to the logger
if not default_logger.handlers:  # Avoid adding duplicate handlers in case the cell is re-executed
    default_logger.addHandler(hdlr=stream_handler)

verbosity: Verbosity = Verbosity.NORMAL
logger: logging.Logger = default_logger

# Example usage
logger.debug(msg="This is a debug message.")
logger.info(msg="This is an info message.")

In [ ]:
comparisons_folder_base_path = pathlib.Path(
    TOPO_LLM_REPOSITORY_BASE_PATH,
    "data/analysis/sample_sizes/",
    "run_general_comparisons/",
    "array_truncation_size=5000/",
    "analysis/twonn/",
)

concatenated_df: DataFrame = load_and_concatenate_saved_dataframes(
    root_dir=comparisons_folder_base_path,
)

columns_to_investigate: list[str] = [
    "data_full",
    "data_subsampling_full",
    "model_partial_name",
]

for column_name in columns_to_investigate:
    print(30 * "=")
    print(
        f"Unique values in column '{column_name = }':",
    )
    print(
        concatenated_df[column_name].unique(),
    )

concatenated_df.info()

In [ ]:
concatenated_df

#### Load the monitoring files containing the losses

In [ ]:
class ModelLossExtractor:
    """Class for extracting the losses of a model over the finetuning checkpoints."""

    def __init__(self, train_loss_file_path: pathlib.Path, eval_loss_file_path: pathlib.Path):
        """Initialize the class."""
        train_loss_df: DataFrame = pd.read_csv(filepath_or_buffer=train_loss_file_path)
        eval_loss_df: DataFrame = pd.read_csv(filepath_or_buffer=eval_loss_file_path)
        self.loss_dfs_container: dict = {
            "train": train_loss_df,
            "eval": eval_loss_df,
        }

    def get_model_losses_over_finetuning_checkpoints(
        self,
        data_full: str,
        data_subsampling_split: str,
        model_partial_name: str,
        language_model_seed: int,
    ) -> pd.DataFrame | None:
        """Extract the losses of a model over the finetuning checkpoints."""
        match data_subsampling_split:
            case "train":
                selected_table = self.loss_dfs_container["train"]
                column_name_split_descriptor = "train"
            case "validation":
                selected_table = self.loss_dfs_container["eval"]
                column_name_split_descriptor = "eval"
            case _:
                return None

        match model_partial_name:
            case "model=model-roberta-base_task-masked_lm_multiwoz21-train-10000-ner_tags_ftm-standard_lora-None_5e-05-constant-0.01-50":
                loss_column_name = f"fresh-microwave-2 - {column_name_split_descriptor}/loss"
            case "model=model-roberta-base_task-masked_lm_one-year-of-tsla-on-reddit-train-10000-ner_tags_ftm-standard_lora-None_5e-05-constant-0.01-50":
                loss_column_name = f"dutiful-snowball-3 - {column_name_split_descriptor}/loss"
            case _:
                return None

        # Create the output dataframe from the selected table and the loss_column_name
        output_table = selected_table[["train/global_step", loss_column_name]]

        output_table = output_table.rename(
            columns={
                "train/global_step": NAME_PREFIXES_TO_FULL_DESCRIPTIONS["ckpt"],
                loss_column_name: "loss",
            },
        )

        return output_table


# Initialize the class
model_loss_extractor = ModelLossExtractor(
    train_loss_file_path=pathlib.Path(
        TOPO_LLM_REPOSITORY_BASE_PATH,
        "data/models/finetuning_monitoring/",
        "wandb_export_2024-11-20T18_47_32.346+01_00_train_loss.csv",
    ),
    eval_loss_file_path=pathlib.Path(
        TOPO_LLM_REPOSITORY_BASE_PATH,
        "data/models/finetuning_monitoring/",
        "wandb_export_2024-11-20T19_02_59.541+01_00_eval_loss.csv",
    ),
)

# # # #
# Test the loss extraction function

# data_full_test, model_partial_name_test = (
#     "data=multiwoz21_spl-mode=do_nothing_ctxt=dataset_entry_feat-col=ner_tags",
#     "model=model-roberta-base_task-masked_lm_multiwoz21-train-10000-ner_tags_ftm-standard_lora-None_5e-05-constant-0.01-50",
# )
data_full_test, model_partial_name_test = (
    "data=one-year-of-tsla-on-reddit_spl-mode=proportions_spl-shuf=True_spl-seed=0_tr=0.8_va=0.1_te=0.1_ctxt=dataset_entry_feat-col=ner_tags",
    "model=model-roberta-base_task-masked_lm_one-year-of-tsla-on-reddit-train-10000-ner_tags_ftm-standard_lora-None_5e-05-constant-0.01-50",
)

data_subsampling_split_test = "train"
# data_subsampling_split_test = "validation"


output = model_loss_extractor.get_model_losses_over_finetuning_checkpoints(
    data_full=data_full_test,
    data_subsampling_split=data_subsampling_split_test,
    model_partial_name=model_partial_name_test,
    language_model_seed=1234,
)

output

### Investigate different model checkpoints

In [ ]:
from topollm.analysis.compare_sampling_methods.make_plots import (
    Y_AXIS_LIMITS,
    create_boxplot_of_mean_over_different_sampling_seeds,
    generate_fixed_params_text,
)


def create_histograms_over_model_checkpoints(
    concatenated_df: pd.DataFrame,
    concatenated_filters_dict: dict,
    base_model_partial_name: str = "model=roberta-base",
    figsize: tuple[int, int] = (24, 8),
    common_prefix_path: pathlib.Path | None = None,
    model_losses_df: pd.DataFrame | None = None,
    verbosity: Verbosity = Verbosity.NORMAL,
    logger: logging.Logger = default_logger,
) -> None:
    """Create histograms over the different model checkpoints for the concatenated dataframe."""
    filtered_concatenated_df: pd.DataFrame = filter_dataframe_based_on_filters_dict(
        df=concatenated_df,
        filters_dict=concatenated_filters_dict,
        verbosity=verbosity,
        logger=logger,
    )

    # # # #
    # Filter for the dataframe with just the base model data

    same_filters_but_for_base_model = concatenated_filters_dict.copy()
    same_filters_but_for_base_model["model_partial_name"] = base_model_partial_name
    # We need to drop the entry for "model_seed", because for the base model this value is empty
    same_filters_but_for_base_model.pop("model_seed")

    filtered_for_base_model_concatenated_df = filter_dataframe_based_on_filters_dict(
        df=concatenated_df,
        filters_dict=same_filters_but_for_base_model,
    )

    # Set all the values in the "model_checkpoint" column to "-1"
    filtered_for_base_model_concatenated_df[NAME_PREFIXES_TO_FULL_DESCRIPTIONS["ckpt"]] = -1

    # # # #
    # Create a dataframe by concatenating the two dataframes
    data_for_checkpoint_analysis_df: DataFrame = pd.concat(
        objs=[filtered_concatenated_df, filtered_for_base_model_concatenated_df],
        ignore_index=True,
    )

    # # # #
    # Group "data_for_checkpoint_analysis_df" by value in 'model_checkpoint' column
    # and make a boxplot of "array_data_truncated_mean" for each group

    fixed_params_text: str = generate_fixed_params_text(
        filters_dict=concatenated_filters_dict,
    )

    for y_min, y_max in Y_AXIS_LIMITS.values():
        if common_prefix_path is not None:
            plot_save_path = pathlib.Path(
                common_prefix_path,
                "plots",
                f"y_{y_min}_{y_max}.pdf",
            )
            raw_data_save_path = pathlib.Path(
                common_prefix_path,
                "raw_data",
                "raw_data.csv",
            )
            aggregated_results_save_path = pathlib.Path(
                common_prefix_path,
                "raw_data",
                "aggregated_results.csv",
            )
        else:
            plot_save_path = None
            raw_data_save_path = None
            aggregated_results_save_path = None

        # TODO: Integrate the plotting of the model losses if available

        create_boxplot_of_mean_over_different_sampling_seeds(
            subset_local_estimates_df=data_for_checkpoint_analysis_df,
            x_column_name=NAME_PREFIXES_TO_FULL_DESCRIPTIONS["ckpt"],
            y_column_name="array_data_truncated_mean",
            fixed_params_text=fixed_params_text,
            figsize=figsize,  # This should be a bit larger than the default size, because we have many checkpoints to show
            y_min=y_min,
            y_max=y_max,
            plot_save_path=plot_save_path,
            raw_data_save_path=raw_data_save_path,
            aggregated_results_save_path=aggregated_results_save_path,
            verbosity=verbosity,
            logger=logger,
        )


def run_checkpoint_analysis_over_different_data_and_models(
    data_full_list_to_process: list[str],
    data_subsampling_split_list_to_process: list[str],
    data_subsampling_sampling_mode_list_to_process: list[str],
    model_partial_name_list_to_process: list[str],
    language_model_seed_list_to_process: list[int],
) -> None:
    """Run the checkpoint analysis over different data and models."""
    for (
        data_full,
        data_subsampling_split,
        data_subsampling_sampling_mode,
        model_partial_name,
        language_model_seed,
    ) in tqdm(
        product(
            data_full_list_to_process,
            data_subsampling_split_list_to_process,
            data_subsampling_sampling_mode_list_to_process,
            model_partial_name_list_to_process,
            language_model_seed_list_to_process,
        ),
        desc="Processing different combinations of data_full, data_subsampling_split, and model_partial_name",
    ):
        concatenated_filters_dict = {
            "data_full": data_full,
            "data_subsampling_split": data_subsampling_split,
            "data_subsampling_sampling_mode": data_subsampling_sampling_mode,
            "data_subsampling_number_of_samples": 10_000,
            "model_partial_name": model_partial_name,
            "model_seed": language_model_seed,
            "data_prep_sampling_method": "random",
            "data_prep_sampling_samples": 150_000,
            NAME_PREFIXES_TO_FULL_DESCRIPTIONS["dedup"]: "array_deduplicator",
            "local_estimates_samples": 60_000,
            "n_neighbors": 128,
        }

        common_prefix_path = pathlib.Path(
            TOPO_LLM_REPOSITORY_BASE_PATH,
            "data",
            "saved_plots",
            "mean_estimates_over_different_checkpoints",
            f"{data_full=}",
            f"{data_subsampling_split=}",
            f"{data_subsampling_sampling_mode=}",
            f"{model_partial_name=}",
            f"{language_model_seed=}",
        )

        create_histograms_over_model_checkpoints(
            concatenated_df=concatenated_df,
            concatenated_filters_dict=concatenated_filters_dict,
            figsize=(22, 8),
            common_prefix_path=common_prefix_path,
            verbosity=verbosity,
            logger=logger,
        )


# # # #
# Select which analysis to run and call the analysis
data_full_list_to_process: list[str] = [
    "data=multiwoz21_spl-mode=do_nothing_ctxt=dataset_entry_feat-col=ner_tags",
    "data=one-year-of-tsla-on-reddit_spl-mode=proportions_spl-shuf=True_spl-seed=0_tr=0.8_va=0.1_te=0.1_ctxt=dataset_entry_feat-col=ner_tags",
]

data_subsampling_split_list_to_process: list[str] = [
    "train",
    "validation",
    "test",
]

data_subsampling_sampling_mode_list_to_process: list[str] = [
    "random",
    "take_first",
]

model_partial_name_list_to_process: list[str] = [
    "model=model-roberta-base_task-masked_lm_multiwoz21-train-10000-ner_tags_ftm-standard_lora-None_5e-05-constant-0.01-50",
    "model=model-roberta-base_task-masked_lm_one-year-of-tsla-on-reddit-train-10000-ner_tags_ftm-standard_lora-None_5e-05-constant-0.01-50",
]

# Note: The "model_seed" column contains type integer values
language_model_seed_list_to_process: list[int] = [
    1234,
    1235,
    1236,
]

# # # #
# Uncomment to run the checkpoint analysis on the full available data
#
# run_checkpoint_analysis_over_different_data_and_models(
#     data_full_list_to_process=data_full_list_to_process,
#     data_subsampling_split_list_to_process=data_subsampling_split_list_to_process,
#     data_subsampling_sampling_mode_list_to_process=data_subsampling_sampling_mode_list_to_process,
#     model_partial_name_list_to_process=model_partial_name_list_to_process,
#     language_model_seed_list_to_process=language_model_seed_list_to_process,
# )

# # # #
# Run checkpoint analysis on example data

run_checkpoint_analysis_over_different_data_and_models(
    data_full_list_to_process=["data=multiwoz21_spl-mode=do_nothing_ctxt=dataset_entry_feat-col=ner_tags"],
    data_subsampling_split_list_to_process=["train"],
    data_subsampling_sampling_mode_list_to_process=["random"],
    model_partial_name_list_to_process=[
        "model=model-roberta-base_task-masked_lm_multiwoz21-train-10000-ner_tags_ftm-standard_lora-None_5e-05-constant-0.01-50",
    ],
    language_model_seed_list_to_process=[1234],
)


### Investigate the influence of the data subsampling method on the results

In [ ]:
from topollm.analysis.compare_sampling_methods.make_plots import Y_AXIS_LIMITS_ONLY_FULL


def log_subsampling_number_of_samples_values(
    filtered_concatenated_df: pd.DataFrame,
) -> None:
    """For every occurence of value in "data_subsampling_number_of_samples", check how many rows are present in the filtered dataframe."""
    data_subsampling_number_of_samples_values = filtered_concatenated_df["data_subsampling_number_of_samples"].unique()

    for data_subsampling_number_of_samples in data_subsampling_number_of_samples_values:
        data_subsampling_number_of_samples_filters_dict = {
            "data_subsampling_number_of_samples": data_subsampling_number_of_samples,
        }

        filtered_concatenated_df_for_number_of_samples: DataFrame = filter_dataframe_based_on_filters_dict(
            df=filtered_concatenated_df,
            filters_dict=data_subsampling_number_of_samples_filters_dict,
        )

        print(
            f"{data_subsampling_number_of_samples = }: {filtered_concatenated_df_for_number_of_samples.shape = }",
        )

    print(
        "Unique data_subsampling_sampling_seed:\n",
        filtered_concatenated_df["data_subsampling_sampling_seed"].unique(),
    )


data_full = concatenated_df["data_full"].unique()[1]
data_subsampling_split = concatenated_df["data_subsampling_split"].unique()[2]
data_subsampling_sampling_mode_list_to_process: str = "random"

model_full = concatenated_df["model_full"].unique()[0]

concatenated_filters_dict = {
    "data_full": data_full,
    "model_full": model_full,
    "data_subsampling_split": data_subsampling_split,
    "data_subsampling_sampling_mode": data_subsampling_sampling_mode_list_to_process,
    "data_prep_sampling_method": "random",
    "data_prep_sampling_samples": 100_000,
    NAME_PREFIXES_TO_FULL_DESCRIPTIONS["dedup"]: "array_deduplicator",
    "local_estimates_samples": 60_000,
    "n_neighbors": 128,
}

filtered_concatenated_df: DataFrame = filter_dataframe_based_on_filters_dict(
    df=concatenated_df,
    filters_dict=concatenated_filters_dict,
)

log_subsampling_number_of_samples_values(
    filtered_concatenated_df=filtered_concatenated_df,
)

print(f"{filtered_concatenated_df.shape = }")

# # # #
# START Additional data cleaning:
# Remove those samples where "array_data.size" is smaller than 30_000

filtered_concatenated_df_cleaned = filtered_concatenated_df[filtered_concatenated_df["array_data.size"] >= 50_000]

# END Additional data cleaning
# # # #


data_for_different_data_subsampling_number_of_samples_analysis_df: DataFrame = filtered_concatenated_df_cleaned

fixed_params_text: str = generate_fixed_params_text(
    filters_dict=concatenated_filters_dict,
)

x_column_name = "data_subsampling_number_of_samples"

for y_min, y_max in Y_AXIS_LIMITS_ONLY_FULL.values():
    # for y_min, y_max in [(6.0, 10.0)]:
    create_boxplot_of_mean_over_different_sampling_seeds(
        subset_local_estimates_df=data_for_different_data_subsampling_number_of_samples_analysis_df,
        plot_save_path=None,  # TODO: Select path
        raw_data_save_path=None,  # TODO: Select path
        x_column_name=x_column_name,
        y_column_name="array_data_truncated_mean",
        seed_column_name="data_subsampling_sampling_seed",
        fixed_params_text=fixed_params_text,
        y_min=y_min,
        y_max=y_max,
        logger=logger,
    )